In [ ]:
from detect import run

In [ ]:
#img = 'data/dwg/images/val'
img = 'data/dwg/images/val/c8fae7c9-3788-4d6c-a767-7e26ed098419.bmp'
#img = 'data/dwg/images/val/11c4395a-51a1-4c1b-8be7-dd61d30a5681.bmp'
run(weights='runs/train/exp/weights/best-s.pt', source=img, nosave=False, save_txt=False, imgsz=512, conf_thres=0.2, iou_thres=0.2, exist_ok=True, line_thickness=1)


In [ ]:
 !python detect.py --weights runs/train/exp2/weights/best.pt --source data/dwg/images/train/0b8c9c96-5361-47ca-8a60-db6ecbb2170c.bmp --imgsz 512 --conf-thres 0.5 --iou-thres 0.2 --exist-ok --save-txt

In [ ]:
from draw_targets_and_predictions_jupyter import PlotEv
PlotEv(weights='runs/train/exp2/weights/best.pt')